# 搭建并使用向量数据库

## 前序embedding模型

ZhipuAIEmbeddings模型

In [1]:
from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)

## 前序数据预处理

批量处理文件夹中所有文件

In [2]:
import os
# from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

# _ = load_dotenv(find_dotenv())


# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

# 下载文件并存储到text
# 作数据清洗
from langchain.document_loaders.pdf import PyMuPDFLoader
import re
# 下载文件并存储到text
texts = []
for loader in loaders:
    texts.extend(loader.load())   

for text in texts:
    pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
    text.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), text.page_content)
    text.page_content = text.page_content.replace('•', '')
    text.page_content = text.page_content.replace(' ', '')



''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 知识库中单段文本长度
CHUNK_SIZE = 512
# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])
split_docs = text_splitter.split_documents(texts)

print(f"---\n文档split成功")


print(f"切分后的文件数量：{len(split_docs)}")

print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")


split_docs[2].page_content

['../data_base/knowledge_path\\pumkin_book\\pumpkin_book.pdf']
---
文档split成功
切分后的文件数量：614
切分后的字符数（可以用来大致评估 token 数）：252076


'有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识\n我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们GitHub的\nIssues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块\n提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的\n话可以微信联系我们（微信号：at-Sm1les）；\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1版）\n最新版PDF获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会'

## 构建Chroma向量库

涉及到的API

In [3]:
# langchain_chroma.vectordbs.Chroma
# class langchain_chroma.vectordbs.Chroma(
#     collection_name: str = 'langchain', 
# embedding_function: Embeddings | None = None, 
# persist_directory: str | None = None, 
# client_settings: Settings | None = None, 
# collection_metadata: Dict | None = None, 
# client: ClientAPI | None = None, 
# relevance_score_fn: Callable[[float], float] | None = None, 
# create_collection_if_not_exists: bool | None = True)


# classmethod from_documents(
#     documents: List[Document], 
# embedding: Embeddings | None = None, 
# ids: List[str] | None = None, 
# collection_name: str = 'langchain', 
# persist_directory: str | None = None, 
# client_settings: Settings | None = None, 
# client: ClientAPI | None = None, 
# collection_metadata: Dict | None = None, 
# **kwargs: Any)

直接调用Chroma.from_documents

In [11]:
# 汇总

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
persist_directory = '../chromatest'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings(model="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=embedding,
        # collection_name="test1", # 如果不指定默认为langchain
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

向量数据库已成功持久化到磁盘。


C:\Users\will\AppData\Local\Temp\ipykernel_16272\3345607360.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


charma服务器版本

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

chroma_client = chromadb.HttpClient(host='localhost', port=8000)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
                model_name="embedding-2"
            )
collection = chroma_client.get_or_create_collection(
    name="test4",
    embedding_function=openai_ef,  # 使用openai
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)
ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    collection.add(documents=split_docs[:20], ids =ids_list )
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

通过chroma数据库add

In [ ]:
from chromadb.utils import embedding_functions
import os
from langchain.vectordbs.chroma import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
# 定义持久化目录
persist_directory = './chroma6'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
                model_name="embedding-2"
            )
collection = client.get_or_create_collection(
    name="test4",
    embedding_function=openai_ef,  # 使用openai
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)
ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    collection.add(documents=split_docs[:20], ids =ids_list )
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

通过chroma数据库add-改良版 每次三个doc，分批次导入

In [ ]:
# 汇总

import os
from langchain.vectordbs.chroma import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
from chromadb import Documents, EmbeddingFunction, Embeddings
# 定义持久化目录
persist_directory = './chroma6'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

# 创建嵌入模型
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
                model_name="embedding-2"
            )

# zhipu_ef = ZhipuAIAdapter(model_name="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 创建集合并配置HNSW参数
# collection = client.create_collection( 
collection = client.get_or_create_collection(
    name="test3",
    embedding_function=openai_ef,
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)

ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    # collection.add(documents=split_docs)

    from itertools import batched
    docs_batches = batched(split_docs[:20], 3)  # 分3个批次
    ids_batches = batched(ids_list, 3)

    for docs, ids in zip(docs_batches, ids_batches):
        collection.add(documents=docs, ids=ids)
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

In [13]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：80


连接chroma数据库

In [14]:
# 数据库路径 ./chroma2

In [15]:
import chromadb
chroma_client  = chromadb.PersistentClient(path="../chromatest")

In [16]:
chroma_client.list_collections()

['langchain']

In [18]:
# 获取一个存在的Collection对象
collection = chroma_client.get_collection("langchain")

In [19]:
collection.count()  #  returns the number of items in the collection.

80

In [20]:
# collection.peek()  # returns a list of the first 10 items in the collection.

In [ ]:
# 获取所有数据（默认不返回嵌入向量）
all_data = collection.get()
# print(all_data)


## 三、向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

In [22]:
question="什么是南瓜书"

In [23]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [24]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
编委会
主编：Sm1les、archwalker、jbb0523
编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226
封面设计：构思-Sm1les、创作-林王茂盛
致谢
特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、spareribs、suncha
--------------
检索到的第1个内容: 
编委会
主编：Sm1les、archwalker、jbb0523
编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226
封面设计：构思-Sm1les、创作-林王茂盛
致谢
特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、spareribs、suncha
--------------
检索到的第2个内容: 
编委会
主编：Sm1les、archwalker、jbb0523
编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226
封面设计：构思-Sm1les、创作-林王茂盛
致谢
特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、spareribs、suncha
--------------


### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [25]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [26]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
编委会
主编：Sm1les、archwalker、jbb0523
编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226
封面设计：构思-Sm1les、创作-林王茂盛
致谢
特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、spareribs、suncha
--------------
MMR 检索到的第1个内容: 
本:1.9.9
发布日期:2023.03
南⽠书
PUMPKINBOOKDatawhale
--------------
MMR 检索到的第2个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们
--------------


# rerank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# Cohere Rerank配置
# https://docs.cohere.com/v2/reference/rerank
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
compressor = CohereRerank(
    client=cohere_client,
    top_n=5  # 保留Top5相关文档
)

# 创建基础的检索器
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

# 创建带重排序的检索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# 5. LLM模型初始化
# from openai import OpenAI 

# llm = OpenAI(
#     api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
#     base_url="https://open.bigmodel.cn/api/paas/v4/"
# ) 
from langchain_community.chat_models import ChatOpenAI
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

# 6. 构建RAG链
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 提示模板设计
template = """基于以下上下文，用中文回答用户问题：
上下文：{context}
问题：{question}
要求：答案需包含引用来源的页码（示例：[P3]）"""
prompt = ChatPromptTemplate.from_template(template)

# 组装RAG链
rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 7. 使用示例
response = rag_chain.invoke("机器学习")
print(response)

# no rerank 和 rerank 的回答结果对比

In [ ]:
# 基础检索器（无重排序）
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

# 构建RAG链
basic_rag_chain = (
    {"context": base_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
# 带重排序的检索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=CohereRerank(client=cohere_client, top_n=5),
    base_retriever=base_retriever
)

# 构建RAG链
rerank_rag_chain = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
# 测试同一问题
question = "机器学习的主要应用场景有哪些？"

# 无Rerank的结果
basic_response = basic_rag_chain.invoke(question)
print("=== 无Rerank的答案 ===")
print(basic_response)

# 带Rerank的结果
rerank_response = rerank_rag_chain.invoke(question)
print("\n=== 带Rerank的答案 ===")
print(rerank_response)

# no rerank 和 rerank 的检索结果对比

In [ ]:
# 在原有代码基础上新增以下对比逻辑

# 执行原始检索（无rerank）
base_docs = base_retriever.invoke("机器学习")
print("=== Rerank前检索结果 ===")
for i, doc in enumerate(base_docs):
    print(f"[文档{i+1}] 相似度分：{doc.metadata.get('score', 'N/A')}")
    print(f"内容摘要：{doc.page_content[:100]}...")
    print(f"元数据：页码={doc.metadata.get('page', 'N/A')}, 来源={doc.metadata.get('source', 'N/A')}\n")

# 执行带rerank的检索
reranked_docs = compression_retriever.invoke("机器学习")
print("\n=== Rerank后检索结果 ===")
for i, doc in enumerate(reranked_docs):
    print(f"[文档{i+1}] 重排序分：{doc.metadata.get('relevance_score', 'N/A')}")
    print(f"内容摘要：{doc.page_content[:100]}...")
    print(f"元数据：页码={doc.metadata.get('page', 'N/A')}, 来源={doc.metadata.get('source', 'N/A')}\n")

# 原有RAG流程保持不变
response = rag_chain.invoke("机器学习")
print("\n=== 最终答案 ===")
print(response)